# 67818 Ex3 - Daniel Busbib

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, balanced_accuracy_score,accuracy_score, make_scorer
from sklearn.metrics import r2_score
import plotly.subplots
import plotly.graph_objects as go
from sklearn import tree
import plotly.express as px
from google.colab import files
import folium, io
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, cross_val_score
from sklearn import linear_model
from scipy.stats import randint
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns


In [2]:
file_name = "ex3.csv"

uploaded = files.upload()
data = pd.read_csv(io.BytesIO(uploaded[file_name]))
# data = pd.read_csv(file_name)

Saving ex3.csv to ex3.csv


# Q1

In [51]:
data1 = data.copy()

In [52]:
data1.columns

Index(['Unnamed: 0', 'incident_id', 'date', 'state', 'city_or_county',
       'address', 'n_killed', 'n_injured', 'congressional_district',
       'gun_stolen', 'gun_type', 'incident_characteristics', 'latitude',
       'location_description', 'longitude', 'n_guns_involved', 'notes',
       'participant_age', 'participant_age_group', 'participant_gender',
       'participant_name', 'participant_relationship', 'participant_status',
       'participant_type', 'state_house_district', 'state_senate_district'],
      dtype='object')

* Feature engineering

In [53]:

data1['month'] = pd.to_datetime(data1['date']).dt.month
data1['year'] = pd.to_datetime(data1['date']).dt.year

data1['gun_stolen'].fillna('', inplace=True)
data1['participant_gender'].fillna('', inplace=True)
data1['participant_age_group'].fillna('', inplace=True)
data1['congressional_district'].fillna(0, inplace=True)

data1['teen_participant'] = data1['participant_age_group'].apply(lambda x: int('Teen' in x))
data1['is_female'] = data1['participant_gender'].apply(lambda x: int('Female' in x))
data1['has_gun_stolen'] = data1['gun_stolen'].apply(lambda x: int('Stolen' in x))

data1['congressional_district'] = data1['congressional_district'].astype(str)


I will use the method I implement from previous exercise to fill all missing `longitude` values, with the mean by state and city/country.

In [54]:
def change(LT):
  STATE = "state"
  CC = "city_or_county"
  table_map = data1[~data1[LT].isna()].groupby([STATE, CC]).mean()[LT]
  table_map_w_cc = data1[~data1[LT].isna()].groupby([STATE]).mean()[LT]
  missing_indices = data1[data1[LT].isna()][LT].index

  def mapF(i):
    state = data1.loc[i][STATE]
    city = data1.loc[i]["city_or_county"]
    return table_map[state][city] if city in table_map[state] else table_map_w_cc[state]

  maplg = pd.Series(map(mapF, missing_indices),
                      index=missing_indices)
  data1.loc[data1[LT].isna(), LT] = maplg
  return

change('longitude')

In [55]:
cd_cols = ['congressional_district_' + dist for dist in list(data1['congressional_district'].unique())]
# st_cols = ['state_' + s for s in list(data1['state'].unique())] 
model_features = ['is_female', 'month', 'year', 'teen_participant', 'has_gun_stolen', 'longitude'] + cd_cols 

* Linear Regression Model

In [56]:
model_data = pd.get_dummies(data1, columns=['congressional_district'])

In [57]:
X_train, X_test, y_train, y_test = train_test_split(model_data[model_features], model_data['n_killed'])

In [58]:
model_1 = LinearRegression()
model_1.fit(X_train, y_train)

LinearRegression()

In [59]:
y_pred_1 = model_1.predict(X_test)
print('mse for linear regression model: ', mean_squared_error(y_pred_1, y_test))
print('mse for linear regression model after converting predict to int type: ', mean_squared_error(y_pred_1.astype(np.int64), y_test))

mse for linear regression model:  0.24407421265405707
mse for linear regression model after converting predict to int type:  0.3196


In [60]:
np.unique(y_pred_1.astype(np.int64))

array([0])

* KNN Model

In [61]:
model_2 = KNeighborsClassifier()
model_2.fit(X_train, y_train)

KNeighborsClassifier()

In [62]:
y_pred_2 = model_2.predict(X_test)
print('mse for knn model: ', mean_squared_error(y_pred_2, y_test))

mse for knn model:  0.34226666666666666


In [63]:
np.unique(y_pred_2)

array([0, 1, 2])

I choose the MSE as a metric for my models, Linear Regression and KNN.

An advantage of MSE with a linear regression model is that it give us a good information about how our model work with prediction. Another advantage is that the MSE is great for ensuring that our trained model has no outlier predictions with huge errors. <br>
An disadvantage is that if our model makes a single very bad prediction, the mse will magnifies the error.

I choose Linear regression because I was interseted to see if this model will give good results, although the `n_killed` feature is a integer feature. The disadvantage is that the model did not care about this, and we need to convert the predict type to int to get the "real" prediction, which probably will give us a higher mse.

For the second model I choose the KNN model, because it is a classification model, and does not include training period. The problem with this model, is that it does not work well with high dimensionality as this will complicate the distance calculating process to calculate distance for each dimension and it is sensitive to noisy and missing data.



From the resutls recorded, the MSE for the linear regression model is lower from the KNN model, even after I convert the predict of LR model to integer type. <br>


# Q2

* Grid search

In [64]:
model = RandomForestClassifier(random_state=10)


params_grid = { 
    'n_estimators': [2, 4, 8, 16, 32],
    'min_samples_leaf': [2, 4, 8, 16],
    'min_samples_split': [2, 4, 8, 16, 32],
}


grid = GridSearchCV(estimator=model, param_grid=params_grid, cv= 5)
grid.fit(X_train, y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=5.



GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=10),
             param_grid={'min_samples_leaf': [2, 4, 8, 16],
                         'min_samples_split': [2, 4, 8, 16, 32],
                         'n_estimators': [2, 4, 8, 16, 32]})

In [65]:
print('Best parameters found in Grid Search', grid.best_params_)
print('Best score in Grid Search', grid.best_score_)

Best parameters found in Grid Search {'min_samples_leaf': 4, 'min_samples_split': 16, 'n_estimators': 16}
Best score in Grid Search 0.7766222222222222


* Random search

In [66]:
model = RandomForestClassifier(random_state=10)

dists = { 
    'n_estimators': randint(2, 256),
    'min_samples_leaf': randint(2, 16),
    'min_samples_split': randint(2, 32)
}

rs = RandomizedSearchCV(estimator=model, param_distributions=dists, cv=5)

rs.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=5.



RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=10),
                   param_distributions={'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f2a15732610>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f2a15732750>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f2a15732690>})

In [67]:
print('Best parameters found in Random Search', rs.best_params_)
print('Best score in Random Search', rs.best_score_)

Best parameters found in Random Search {'min_samples_leaf': 7, 'min_samples_split': 4, 'n_estimators': 137}
Best score in Random Search 0.7764


# Q3a

In [74]:
features  = ['is_female', 'month', 'year', 'teen_participant', 'has_gun_stolen', 'longitude']

In [75]:
X_train, X_test, y_train, y_test = train_test_split(data1[features], data1['n_killed'], test_size=0.3)
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_1 = lr.predict(X_test)
print('accuracy score: ', accuracy_score(y_test, y_pred_1.astype(np.int64)))
print('mse: ', mean_squared_error(y_test, y_pred_1))

accuracy score:  0.774
mse:  0.235019375384435


In [76]:
px.bar(x=X_train.columns, y=lr.coef_, title='linear regression model - feature importance', labels={'x': 'Feature', 'y':'coef value'})

As we can see, the `is_female` feature is the most important features, although 
`has_gun_stolen` has also a high abs coef value.

# Q3b

In [77]:
# model without is_female
lr_1 = LinearRegression().fit(X_train.drop(['is_female'], axis= 1), y_train)
px.bar(x=X_train.columns.drop(['is_female']), y=lr_1.coef_, title='linear regression model - feature importance', labels={'x': 'Feature', 'y':'coef value'})

In [78]:
y_pred = lr_1.predict(X_test.drop(['is_female'], axis= 1))
print('accuracy score: ', accuracy_score(y_test, y_pred.astype(np.int64)))
print('mse: ', mean_squared_error(y_test, y_pred))

accuracy score:  0.774
mse:  0.2400883934852271


In [79]:
best_4 =  ['is_female', 'teen_participant', 'has_gun_stolen', 'longitude']
lr_2 = LinearRegression().fit(X_train[best_4], y_train)

In [80]:
px.bar(x=X_train[best_4].columns, y=lr_2.coef_, title='linear regression model - feature importance', labels={'x': 'Feature', 'y':'coef value'})

In [81]:
y_pred = lr_2.predict(X_test[best_4])
print('accuracy score: ', accuracy_score(y_test, y_pred.astype(np.int64)))
print('mse: ', mean_squared_error(y_test, y_pred))

accuracy score:  0.774
mse:  0.23504994817323965


As I can see from the results, removing the best feature does not very helped the model and increase a little bit the MSE.<br>
I am not very surprised that the model with the most important features did not change a lot, and I can think it's because the model already made decision by those features in the first model with all the features(q3a).